In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm
from statsmodels.stats.proportion import confint_proportions_2indep
from statsmodels.stats.proportion import proportions_ztest
# import the beta function from scipy.special
from scipy.special import beta as beta_function
from scipy.stats import beta as beta_dist


# A/B Test Methodologies: Null Hypothesis Significance Testing vs. Bayesian Approaches

When evaluating new user experiences (UX) — such as launching **passkeys** and measuring their impact on abandonment rates — we need a way to decide whether a new design is better, worse, or equivalent to the current one.  
This notebook compares two major approaches:

- **Null Hypothesis Significance Testing (NHST)** — the long-standing statistical framework, widely used but often conceptually tricky and difficult to interpret in practical decision-making.
- **Bayesian methods** — increasingly popular because they offer more flexibility and produce results that are often easier to interpret directly when deciding actions (e.g., when to shift more traffic from a control to a new variant).

---

## Test Setup: Control Group vs. Variants

We assume an existing digital identity creation flow with a **completion rate of ~20%** (meaning ~80% of users abandon).  
Our test design:

- Keep **90%** of traffic on the current experience as the **control group**.
- Send the remaining traffic to one or more **variants** $A, B, C$.

The test will be conducted in 2 steps, first we need to determine that each new experience is **no worse** than the current one, accepting small degradaition as unavoidable as we are adding more pages and clicks  — then after establishing we haven't degraded the experience, shifting more traffic to the variants and decidng which is the better-performing variant.

The type of A/B test — where the first goal is to ensure a new design does **not degrade** the experience — is called a **non-inferiority test** (explained below).


## Null Hypothesis Significance Testing (NHST)

At a high level, the **NHST** workflow is:

1. **Assume what you *don’t* want to see** — this is the **null hypothesis**.  
   - Example in medicine: *“the drug has no effect.”*  
   - Example here: *“the new experience increases abandonment.”*
2. **Run the experiment** and compute a test statistic.
3. **Ask:** *If the null were true, how likely is it that we would observe a result at least this extreme?*  
   - If that probability (the **p-value**) is very low — e.g., below a conventional threshold such as 5% — we **reject the null**.

Two immediate caveats:
- Rejecting the null does **not** prove the opposite is true; it only says the data would be unlikely *if* the null were correct.
- “Unlikely enough” is arbitrary — thresholds like 5% are conventions, not laws of nature.

A key point: NHST computes **$P(\text{data} \mid \text{hypothesis})$**.  
Later we’ll see that the Bayesian approach instead computes **$P(\text{hypothesis} \mid \text{data})$** — a fundamentally different quantity.

---

### Modeling Conversion as Random Variables

The abandonment or conversion of a UX flow can be modeled with **Bernoulli random variables**:

- $X_C$ for the control experience
- $X_A$ for a new variant $A$

A Bernoulli variable takes only two values: success/failure, convert/abandon, etc.  
Each user who sees a page gives one draw from one of these variables.

We assume both have the same codomain:

$$
\mathcal{X}_C = \mathcal{X}_A = \{0,1\}
$$

where **1 = convert** (user finishes the intended action, e.g., creating a passkey) and **0 = abandon**.  
Technical failures are treated as *success* here because the user attempted the action.

---

### Sample Proportions

NHST usually works with **sample proportions**, the average of $n$ Bernoulli draws:

$$
\hat{p}_C = \frac{1}{n}\sum_{i=1}^n X_{C_i},
\quad
\hat{p}_A = \frac{1}{n}\sum_{i=1}^n X_{A_i}.
$$

Each $\hat{p}$:

- Is a random variable taking values $\{0,\tfrac1n,\tfrac2n,\ldots,1\}$.
- Is also an **estimator** of the true expected value $p = E[X]$.  
  By the law of large numbers, $\hat{p} \to p$ as $n$ grows.

(Statisticians use a “hat” to denote an estimator.)

Formally, an estimator maps $n$ realizations of $X$ ($\mathcal{X}^n$) to a real number:

$$
\hat{p}: \mathcal{X}^n \to [0,1].
$$

Because it is the mean of $n$ Bernoulli variables, $\hat{p}$ follows a **binomial** distribution that becomes approximately **Gaussian** when $n$ is large.

---

### Variance and Standard Deviation of a Sample Proportion

For a single Bernoulli $X$:  
$$
\mathrm{Var}(X) = p(1-p).
$$

For the sample proportion:
$$
\mathrm{Var}\!\left(\tfrac1n \sum_{i=1}^n X_i\right)
= \tfrac1{n^2} n p(1-p)
= \tfrac{p(1-p)}{n}.
$$

$$
\boxed{\mathrm{Var}(\hat{p}) = \frac{p(1-p)}{n}}
$$

The square root of this variance is the **standard error** — a quantity we’ll use later.

---

### Difference in Proportions

We’ll often look at the **difference** between variant and control:

$$
\hat{\Delta} = \hat{p}_A - \hat{p}_C
$$

This estimates the true difference

$$
\Delta = p_A - p_C.
$$

---

### Hypotheses

- **Null Hypothesis $H_0$** — the “bad” scenario we want to reject:  
  the new UX **degrades** conversion by at least some small amount $\epsilon$ we consider unacceptable (e.g., $3\%$):

  $$
  H_0: E[\Delta] \le -\epsilon
  $$

- **Alternative Hypothesis $H_1$** — the new UX is **not worse** than control (possibly better):

  $$
  H_1: E[\Delta] > -\epsilon
  $$

- **Boundary Hypothesis** — used in test construction:  
  assume the difference is exactly at the acceptable degradation limit:

  $$
  E[\Delta] = -\epsilon
  $$


#### Numerical Example

For a concrete example, we define the following counts and quantities for each experience:

- $n_C$ — number of visitors in the **control** group  
- $x_C$ — number of **conversions** observed in the control group

- $n_A$ — number of visitors in the **variant** group  
- $x_A$ — number of **conversions** observed in the variant group

- $\hat{\Delta}_{\mathrm{obs}}$ — the **observed difference** in conversion proportions between variant and control

- $\epsilon$ — the **acceptable degradation margin**, i.e., the smallest decrease in conversion we are willing to tolerate for the new variant




In [46]:
control_group_conversion_rate = 0.2 # based on historical data
nC = 7000
xC_observed = nC * control_group_conversion_rate
nA = 150
xA_observed = 33
epsilon = 0.03
alpha = 0.05
hatpC_observed = xC_observed / nC
hatpA_observed = xA_observed  / nA
hatDelta_observed = hatpA_observed - hatpC_observed

print(f"Realization of difference in conversion rate estimator: {hatDelta_observed:.4f}")
print(f"Control group realization of conversion rate estimator: {hatpC_observed:.4f}")
print(f"Treatment group realization of conversion rate estimator: {hatpA_observed:.4f}")

Realization of difference in conversion rate estimator: 0.0200
Control group realization of conversion rate estimator: 0.2000
Treatment group realization of conversion rate estimator: 0.2200


### Standard Deviation of the Estimator $\hat{\Delta}$ (a.k.a. Standard Error in Frequentist Statistics)

In NHST, the first step is to **estimate the standard deviation** of the estimator $\hat{\Delta}$ (often called the **standard error**, SE).  
We then compare the observed difference in proportions from the experiment to this estimated variability to decide whether the observed effect is “far enough” from what we would expect under the null hypothesis $H_0$.

This is a key pain point for NHST:

- We **do not know** the true standard deviation — it depends on the unknown underlying conversion probabilities.
- Frequentist methods therefore use the **plug-in principle**: estimate the unknown variance by “plugging in” the sample estimates (the data you just observed).

But note the circularity:

1. We want to know if the data are unusual under $H_0$.
2. To measure “unusual,” we need the standard error assuming $H_0$.
3. SE depends on the unknown true rates, so we **plug in** $\hat{p}$ (from the data!).
4. We then use this data-derived SE to judge whether the data are unusual.

It’s like saying: *“Use my one measurement to tell me how variable my measurements are, then use that to decide if my measurement is surprising.”*

Frequentists accept this because:

- **Long-run frequency view:** if we repeated the procedure many times, it would have correct average properties.
- **Pragmatism:** it’s computable from one experiment.
- **Simulation evidence:** works “reasonably well” for moderate $n$, though “reasonably well” is debatable (see the *replication crisis*).
- **Framework limitation:** in classical stats, parameters are fixed unknowns, so no natural way to treat them as random.

---

### Common Plug-In Approaches (a.k.a. “Standard Error Hacks”)

#### 1. Wald **Pooled** Standard Error (for “No Effect” Null Hypothesis)

If the null is **no effect** ($p_A = p_C$), we can pool data from control and variant, because under $H_0$ they are assumed to come from the same distribution.

Realizations of sample proportions are:

$$
\hat{p}_A = \frac{x_A}{n_A}, \qquad
\hat{p}_C = \frac{x_C}{n_C}.
$$

If $H_0$ were true (no difference), a pooled estimator is:

$$
\hat{p}_{\text{pool}} = \frac{x_A + x_C}{n_A + n_C}.
$$

Variance of the difference between two independent proportions is the sum of their variances:

$$
\mathrm{Var}(\hat{p}_A - \hat{p}_C)
= \mathrm{Var}(\hat{p}_A) + \mathrm{Var}(\hat{p}_C).
$$

Plugging in the pooled estimate:

$$
\mathrm{Var}(\hat{p}_A - \hat{p}_C)
= \hat{p}_{\text{pool}}(1-\hat{p}_{\text{pool}})
\left(\frac{1}{n_A}+\frac{1}{n_C}\right).
$$

So the **pooled standard error** is:

$$
\boxed{\text{WaldPooled SE} =
\sqrt{\hat{p}_{\text{pool}}(1-\hat{p}_{\text{pool}})
\left(\frac{1}{n_A}+\frac{1}{n_C}\right)} }
$$

This is the classic **standard error of the difference between two independent proportions**.

Once SE is computed, we calculate a **z-score** (number of SEs the observed difference is away from the null value 0):

$$
z = \frac{\hat{p}_A - \hat{p}_C}{\text{SE}}.
$$

---

#### 2. Wald **Unpooled** Standard Error (for Non-Inferiority)

If the null is **non-inferiority** (allowing a margin $\epsilon$), we **cannot** assume $p_A = p_C$, so we don’t pool.

Instead we sum the individual variances (using the plug-in trick separately for each group):

$$
\widehat{\text{WaldUnpooled SE}} =
\sqrt{\frac{\hat{p}_A(1-\hat{p}_A)}{n_A} +
      \frac{\hat{p}_C(1-\hat{p}_C)}{n_C}}.
$$

Ideally, the true $p_A$ and $p_C$ should be used here, but we don’t know them — so we substitute $\hat{p}_A$ and $\hat{p}_C$.  
This works but can be inaccurate if sample sizes are small or rates are extreme.

---

#### 3. Newcombe / Score-Based (Wilson)

- **Better coverage** than Wald, especially with imbalanced sample sizes or very high/low $p$.
- Still closed-form and relatively easy to compute.
- Still uses plug-in estimates and suffers from the same “single-sample” limitation.

*(Not detailed here — but recommended over Wald when possible.)*

---

#### 4. Miettinen–Nurminen

- Widely used in **clinical trials** and regulated industries (e.g., FDA guidance).
- Provides improved accuracy for non-inferiority tests.
- However: mathematically complex, still plug-in based, and still fundamentally relies on one sample.

---

### Summary

Frequentist methods **must estimate variance from the data itself** — leading to circularity and potential miscalibration, especially for small samples or edge cases.  
Later we’ll see how Bayesian methods avoid this by modeling uncertainty about the true conversion rates directly.


Numerical examples

In [ ]:
pooled_proportion = (xC_observed + xA_observed) / (nC + nA)
wald_pooled_SE = (pooled_proportion * (1 - pooled_proportion) * (1/nC + 1/nA))**0.5
print(f"Wald Pooled Standard Error: {wald_pooled_SE:.4f}")
wald_unpooled_SE = ((hatpC_observed * (1 - hatpC_observed) / nC) + (hatpA_observed * (1 - hatpA_observed) / nA))**0.5
print(f"Wald Unpooled Standard Error: {wald_unpooled_SE:.4f}")

#### Probability of False Positive, *p*-value, Significance Level $\alpha$ (sometimes called “confidence”), and Critical Value

Once we have an estimate of the standard error $SE$ (the standard deviation of $\hat{\Delta}$), NHST assumes a **sampling distribution** for the estimator under the null hypothesis $H_0$.

The idea is:

- If we know (or assume) the **mean** and **standard deviation** of $\hat{\Delta}$ under $H_0$,  
- we can model it with a known probability distribution and calculate how likely any observed result is.

Although the true process is discrete (binomial), in practice we often approximate it by a **normal (Gaussian)** distribution. This is mathematically simpler and is a good approximation for moderate or large sample sizes.

---

##### Using the “Boundary” Case

The null hypothesis is technically an inequality:

$$
H_0: E[\Delta] \le -\epsilon.
$$

But to get a single distribution to work with, we typically use the **boundary value**:

$$
E[\Delta] = -\epsilon.
$$

Why?  
- The boundary is the **least favorable** case for rejecting $H_0$.  
- If we can reject $H_0$ at $E[\Delta] = -\epsilon$, then we would also reject any stronger null (mean further left).  
- This makes the test conservative.

So under $H_0$, we model $\hat{\Delta}$ as:

$$
\hat{\Delta} \sim N(\mu, \sigma)
$$

with

$$
\mu = -\epsilon, \qquad \sigma = SE.
$$

---

##### Computing the *p*-Value

The **p-value** is the probability (under $H_0$) of observing a result **as extreme or more extreme** than what we got, in the direction of the alternative $H_1$.

In this one-sided non-inferiority test, that means the right tail probability:

$$
p\text{-value} = P_{H_0}\big[\hat{\Delta} \ge \hat{\Delta}_{\text{obs}}\big]
= \int_{\hat{\Delta}_{\text{obs}}}^{+\infty} 
\frac{1}{\sqrt{2\pi}\,\sigma}
\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)\,dx.
$$

This tail integral is the **survival function** of the normal distribution.

Using the standard normal CDF $\Phi$:

$$
p\text{-value} = 1 - \Phi\!\left(\frac{\hat{\Delta}_{\text{obs}}-\mu}{\sigma}\right).
$$

---

##### Significance Level $\alpha$ and Critical Value

- We choose a **significance level** $\alpha$ (often $0.05$).  
- If $p\text{-value} \le \alpha$, we reject $H_0$ — our result is unlikely under the null.

The **critical value** $c$ is the smallest observed difference that would lead to rejection at level $\alpha$. It is obtained by inverting the right-tail probability:

$$
c = \mu + \sigma \,\Phi^{-1}(1 - \alpha).
$$

Any observed $\hat{\Delta}_{\text{obs}} \ge c$ yields $p\text{-value} \le \alpha$ and thus rejects $H_0$.

---

Below we’ll compute the p-value and critical value explicitly and visualize how the right tail behaves.


In [ ]:
SE_H0 = wald_unpooled_SE
from scipy.stats import norm

mu_H0 = -epsilon    # mean
sigma_HO = SE_H0  # standard deviation
x = hatDelta_observed  # value to evaluate

# Survival function P(X > x)
p_value = norm.sf(x, loc=mu_H0, scale=sigma_HO)
print(f'p-value (one-sided): {p_value:.4f}')

critical_value = norm.isf(alpha, loc=mu_H0, scale=sigma_HO)
print(f"Critical value for p-value={alpha:.4f}: {critical_value:.4f}")



In [ ]:
# Plot the Gaussian N(mu, sigma) and shade the right-tail area beyond x

# Use previously defined values; recompute to be robust
SE_H0 = wald_unpooled_SE
mu_H0 = -epsilon
sigma_HO = SE_H0
x0 = hatDelta_observed

# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H0 - 6 * sigma_HO
right = mu_H0 + 6 * sigma_HO
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H0, scale=sigma_HO)

# Right-tail probabilityy
p = norm.sf(x0, loc=mu_H0, scale=sigma_HO)

# Critical value at significance alpha (one-sided)
crit_x = norm.ppf(1 - alpha, loc=mu_H0, scale=sigma_HO)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density")

# Shade right tail
mask = xs >= x0
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, label=f"Right tail p = {p:.4g}")

# Vertical line at observed x
ax.axvline(x0, color="C1", ls="--", lw=1.5, label=f"observed delta = {x0:.4f}")

# Vertical line at critical value
ax.axvline(crit_x, color="C2", ls="-.", lw=1.5, label=f"critical value c = {crit_x:.4f}")

# Vertical line at the mean for the null hypothesis H0
ax.axvline(-epsilon, color="k", ls=":", lw=1.5, label=f"mean under H0 = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H0:.4f}, σ={sigma_HO:.4f}) — Right-tail beyond x")
ax.set_xlabel("Δ (difference in proportions) under the null H0")
ax.set_ylabel("Probability density under H0")
ax.legend(loc="best")
ax.grid(True, ls=":", alpha=0.5)
plt.tight_layout()
# plt.show(), display handled by notebook



#### traditional presentation of the same conmputation using z-scores

The traditional ways for NHST to compute the p value is not to use the measurment $\hat{\Delta_{obs}}$ directly but to normalize it to a standard normal and centered on zero and on standard deviation 1 N(0,1). This mormalization is done by transforming the measurment into a so called z-score:

The the z-score is just the way you can rescale an arbitrary Gaussian to the Normal by substracting the meand and dividing by the standard deviation $Z = \frac{X-\mu}{\sigma}$

$Z_{NI} = \frac{\hat{\Delta} - (E(\Delta_{H_{boundary}}))}{SE}$ 

$= \frac{\hat{\Delta} - (-\epsilon)}{SE}$ 

$\boxed{Z_{NI}= \frac{\hat{\Delta} + \epsilon}{SE}}$

Then  $\frac{\hat{\Delta} + \epsilon}{SE}$ is a normal N(0,1) and 

$p(\hat{\Delta} \ge \hat{\Delta(w)})$

$ = p(\frac{\hat{\Delta} + \epsilon}{SE} \ge \frac{\hat{\Delta(w) +\epsilon}}{SE })$

Then we can compute the survial function but this timne on a standard normal and from the z-score value

$\int_{ Z_{NI} }^{+\infty} \frac{1}{\sqrt{2 \pi}} e^{-\frac{1}{2}z^2} dz$

which give the exact same results:



In [ ]:
zni = (hatDelta_observed + epsilon) / SE_H0
p_zni = norm.sf(zni)
print(f"z_NI: {zni:.4f}, p-value: {p_zni:.4f}")

#### False positive
In this NHST the pvalue is also the probability of a false positive that is the probabilityy of rejecting the null hopythisis (saying htere is no degradation) while there is acxtually a degradation. So it is setup our risk of make the wrong decision is 5%

#### Probability of having false negative a.k.a type 2 error , Power and sample size required to get to signficance under those conditions

Conversely , "False negative" in the context of a non inferiority test is the probabilityy of failing to reject given the fact that H1 is true, meaning we fail to clear the non inferiority test, while the new UX is actually non inferior (i.e. as good or better than the old one). So we have the same issue as for the false poistive computation, we have to pick a value for what expected value for the $\Delta$ but this time under H1. As we cannot really work with a range of expecgted value for $\Delta$  we need a single value to fix the gaussian we are going to integrate. Here possible choice is to also pick a boundary which is $E(\Delta) = E(p_C)$ so whatever the control group mean was or is (depending on how much historical data we have). This is usally called "minimum effect size we want to detect" as is really a business decision, it is a bit arbitrary. Pickig our baseline is just one option

To model the estimator under the alternative hypothesis H1, we need to pick another expected value for the $\Delta$, since this is for H1 which means the same or better, we pick the bare minimum we need to make that statement that is "the same" and an expected value of zero for the Delta. Under these condition we can pool the samples to get an estimator of the variance and standard error as we are assuming they are distributed indentically

$SE | H1 = WaldPooled SE = \sqrt{\hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool})) (\frac{1}{n_C} + \frac{1}{n_A})}$

With that in place we compute the probability of the estimator coming up "higher" but with the same critical value as it was established ahead of time with the significance level alpha.

The Beta is the probability that we observed something lower and up to the critical value defined by inverting the p-value but under H1, meaning the distribution is centered on the H1 expected value, in our case it is zero.

This computation will give us whatever it gives us. If we want to TARGET a Power of 0.8 (which means β = 0.2, or 20% Type II error rate), we can compute the sample sizes (implied in SE) that will make us reach that level.



In [ ]:
SE_H1 = wald_pooled_SE
mu_H1 = 0
sigma_H1 = SE_H1
x = critical_value
beta = norm.cdf(x, loc=mu_H1, scale=sigma_H1)
print(f"probabilityy of false negative a.k.a β a.k.a type 2 errors,  at critical value : {beta:.4f}")
power = 1 - beta
print(f"Power (1 - β): {power:.4f}")


In [ ]:
# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H1 - 6 * sigma_H1
right = mu_H1 + 6 * sigma_H1
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H1, scale=sigma_H1)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density under H₁")

# Shade left tail (Type II error region)
mask = xs <= critical_value
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, 
                label=f"Type II error β = {beta:.4g}")

# Vertical line at critical value
ax.axvline(critical_value, color="C2", ls="-.", lw=1.5, 
           label=f"critical value c = {critical_value:.4f}")

# Vertical line at observed delta
ax.axvline(hatDelta_observed, color="C1", ls="--", lw=1.5, 
           label=f"observed delta = {hatDelta_observed:.4f}")

# Vertical line at the mean under H1
ax.axvline(mu_H1, color="k", ls=":", lw=1.5, 
           label=f"mean under H₁ = {mu_H1:.4f}")

# Vertical line at the H0 boundary (for reference)
ax.axvline(-epsilon, color="gray", ls=":", lw=1.5, alpha=0.7,
           label=f"H₀ boundary = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H1:.4f}, σ={sigma_H1:.4f}) — Type II Error (β) and Power")
ax.set_xlabel("Δ (difference in proportions) under H₁")
ax.set_ylabel("Probability density under H₁")
ax.legend(loc="best", fontsize=9)
ax.grid(True, ls=":", alpha=0.5)

# Add text annotation for power
power_text = f"Power = 1 - β = {power:.4f}"
ax.text(0.98, 0.95, power_text, transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()

### Bayesian Approach

In contrast to NHST the Bayesian approach is conceptually simpler, instead of artificially considering only 2 possible expected value boundaries for $\Delta$ one for H0 and one for H1, we assume we just don't know that $E(\Delta)$ can take any value between 0 and 1 , and this is just itself random variable but one that we can quantify accross all possible world

#### Using Beta distribution as a tool to model our 'pior belief"

For kind of Bernouilli trial we are dealin wiht (i.e. success/failure, convert/abandom etc...) the most convienent model for our prior belive is what is know as a Beta distribution, note that the anme "Beta" is unfortunate it as it sonds like the Beta in NHST, but it  has nothint ot with it, is is a probability distribution depending on 2 parameters which depening on how they are set can model "we know nothing a piorir" a.k.a an uniformative prior, or  "we don't know much but just that the coversion rate should be aropund 20% but we may be surprised" to  very strong prior based on tons of historical data "the conversion shold be ver close to 20%"

Note tha the Beta distribution is deined on the interval [0,1] as it is modeling a probability, so that'teh probability of a probability (meta probability)

Here are  few example of priors that coud be used, in cour case we will try with a totally unfornative priot (we know nothing) and weakly one (shoub be around 17% but we are not sure).

In [ ]:
x = np.linspace(0, 1, 1000)

# Four cases:
cases = [
    ("Uninformative (flat)", ("single", (1, 1))),
    ("Weakly informative (centered, high entropy)", ("single", (3, 12))),   # mean=0.2
    ("Strong conviction (centered, low entropy)", ("single", (200, 800))),  # mean=0.2
    ("Bi-modal (mixture of Betas)", ("mixture", ((5, 20, 0.5), (20, 5, 0.5))))
]

fig, axes = plt.subplots(2, 2, figsize=(10, 7), sharex=True, sharey=True)

for ax, (title, (kind, params)) in zip(axes.ravel(), cases):
    if kind == "single":
        a, b = params
        y = beta_dist.pdf(x, a, b)
        mean = a / (a + b)
        ci_low, ci_high = beta_dist.ppf([0.025, 0.975], a, b)

        ax.plot(x, y, color="C0", lw=2)
        ax.fill_between(x, y, color="C0", alpha=0.12)
        ax.axvline(mean, color="k", ls=":", lw=1.2, label=f"mean={mean:.3f}")
        ax.axvline(ci_low, color="C1", ls="--", lw=1, label="95% CI")
        ax.axvline(ci_high, color="C1", ls="--", lw=1)
        ax.set_title(f"{title}\nBeta(α={a}, β={b})", fontsize=10)
        ax.legend(fontsize=8, loc="best")
    else:
        # Mixture of two Betas: (a1, b1, w1), (a2, b2, w2)
        (a1, b1, w1), (a2, b2, w2) = params
        y = w1 * beta_dist.pdf(x, a1, b1) + w2 * beta_dist.pdf(x, a2, b2)
        m1, m2 = a1 / (a1 + b1), a2 / (a2 + b2)
        mix_mean = w1 * m1 + w2 * m2

        ax.plot(x, y, color="C0", lw=2, label="mixture pdf")
        ax.fill_between(x, y, color="C0", alpha=0.12)
        # Component means
        ax.axvline(m1, color="C2", ls="--", lw=1, label=f"mean₁={m1:.2f}")
        ax.axvline(m2, color="C3", ls="--", lw=1, label=f"mean₂={m2:.2f}")
        # Mixture mean
        ax.axvline(mix_mean, color="k", ls=":", lw=1.2, label=f"mixture mean={mix_mean:.2f}")
        ax.set_title(f"{title}\n{w1:.1f}·Beta({a1},{b1}) + {w2:.1f}·Beta({a2},{b2})", fontsize=10)
        ax.legend(fontsize=8, loc="best")

for ax in axes[-1]:
    ax.set_xlabel("p")
for ax in axes[:, 0]:
    ax.set_ylabel("density")

fig.suptitle("Four Beta Priors: flat, weakly centered, strongly centered, bi-modal", fontsize=12)
plt.tight_layout(rect=[0, 0, 1, 0.95])

#### Conservative approach assuming we know nothing (a.k.a non informative prior)

For the Variant A, assuming we start with an non informative prior meaning that we have no idea, p_A could range from 0 to 1 with equal probabilityy model by the beta function $Beta(1,1)$  
 
after n trials and k success the posterior probabilityy distribution due to the property of the Beta function is
 
$Beta(xA+1, n_A - xA+1)$ 

This derived through Bayes Theorem and how the Beta function can be integrated

The expected value for $E(Beta(\alpha,\beta)) = \frac{\alpha}{\alpha + \beta}$

which for a bayesian posterior becomes

$E(Beta(xA+1,nA−xA+1)) = \frac{xA+1}{xA+1 + nA−xA+1} = \frac{xA+1}{nA+2}$

In [ ]:
expected_value_posterior = (xA_observed + 1) / (nA + 2)
print(f"Expected value of posterior distribution for p_A: {expected_value_posterior:.4f}")

Now let's compute the credible intervals and visualize the prior and posterior distributions.

In [ ]:
# Posterior parameters
alpha = xA_observed + 1
beta_param = nA - xA_observed + 1

# Ensure we use the beta distribution from scipy.stats
from scipy.stats import beta as beta_dist

# Compute 95% credible interval (2.5th and 97.5th percentiles)
p_L = beta_dist.ppf(0.025, alpha, beta_param)
p_U = beta_dist.ppf(0.975, alpha, beta_param)

# Output the result
print(f"95% Credible Interval for p: [{p_L:.4f}, {p_U:.4f}]")

In [ ]:
# compute the 99% credible interval (1th and 99th percentiles)
p_L_99 = beta_dist.ppf(0.005, alpha, beta_param)
p_U_99 = beta_dist.ppf(0.995, alpha, beta_param)
# Output the result
print(f"99% Credible Interval for p: [{p_L_99:.4f}, {p_U_99:.4f}]")


In [ ]:
# Visualize non-informative prior vs posterior
x_range = np.linspace(0, 0.5, 1000)
prior_noninformative_pdf = beta_dist.pdf(x_range, 1, 1)  # Beta(1,1) - uniform
posterior_noninformative_pdf = beta_dist.pdf(x_range, alpha, beta_param)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_range, prior_noninformative_pdf, 'b--', lw=2, label='Prior: Beta(1, 1) - Non-informative')
ax.plot(x_range, posterior_noninformative_pdf, 'r-', lw=2, label=f'Posterior: Beta({alpha:.1f}, {beta_param:.1f})')

# Mark the non-inferiority boundary
ax.axvline(control_group_conversion_rate - epsilon, color='k', ls=':', lw=1.5, 
           label=f'Non-inferiority boundary = {control_group_conversion_rate - epsilon:.2f}')

# Mark the control conversion rate
ax.axvline(control_group_conversion_rate, color='g', ls=':', lw=1.5, 
           label=f'Control conversion rate = {control_group_conversion_rate:.2f}')

# Shade the 95% credible interval
mask = (x_range >= p_L) & (x_range <= p_U)
ax.fill_between(x_range[mask], posterior_noninformative_pdf[mask], alpha=0.3, color='red', 
                label='95% Credible Interval')

ax.set_xlabel('Conversion rate p_A', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Non-informative Prior vs Posterior Distribution', fontsize=14)
ax.legend(loc='best')
ax.grid(True, ls=':', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Compute probability that variant is non-inferior using non-informative prior
prob_non_inferior_noninformative = 1 - beta_dist.cdf(control_group_conversion_rate - epsilon, 
                                                       alpha, beta_param)
print(f"Probability that variant is non-inferior (non-informative prior): {prob_non_inferior_noninformative:.4f}")
print(f"This means there's a {prob_non_inferior_noninformative*100:.2f}% probability that the variant conversion rate is above {control_group_conversion_rate - epsilon:.2f}")

#### Weakly Informative prior approach using historical data

Instead of using a non-informative prior Beta(1,1), we can incorporate our historical knowledge. We know the control group has a conversion rate of 0.2, and we want to test for non-inferiority with margin epsilon = 0.03. 

For the variant, we'll use a prior centered at 0.2 - 0.03 = 0.17 (the boundary of non-inferiority), but with high entropy to reflect our uncertainty. A Beta distribution with mean μ and relatively small α and β values will have high entropy while still being informative about the center.

For a Beta(α, β) distribution:
- Mean: $\mu = \frac{\alpha}{\alpha + \beta}$
- To get high entropy (concave distribution), we want small α and β values (both < 1 makes it U-shaped, but values around 2-5 give high entropy while remaining unimodal)

Let's use α = 3, and solve for β to get mean = 0.17:
- $0.17 = \frac{3}{3 + \beta}$
- $\beta = \frac{3}{0.17} - 3 \approx 14.65$

After observing the data, the posterior will be Beta(xA + α, nA - xA + β)

In [ ]:
# Informative prior parameters
target_prior_mean = control_group_conversion_rate - epsilon  # 0.17
alpha_prior = 3  # Small value for high entropy
beta_prior = (alpha_prior / target_prior_mean) - alpha_prior  # Solve for beta given mean

print(f"Prior: Beta({alpha_prior:.2f}, {beta_prior:.2f})")
print(f"Prior mean: {alpha_prior / (alpha_prior + beta_prior):.4f}")
print(f"Prior variance: {(alpha_prior * beta_prior) / ((alpha_prior + beta_prior)**2 * (alpha_prior + beta_prior + 1)):.6f}")

# Posterior parameters after observing data
alpha_posterior = xA_observed + alpha_prior
beta_posterior = (nA - xA_observed) + beta_prior

print(f"\nPosterior: Beta({alpha_posterior:.2f}, {beta_posterior:.2f})")
posterior_mean = alpha_posterior / (alpha_posterior + beta_posterior)
print(f"Posterior mean: {posterior_mean:.4f}")

# Compute 95% credible interval
p_L_informative = beta_dist.ppf(0.025, alpha_posterior, beta_posterior)
p_U_informative = beta_dist.ppf(0.975, alpha_posterior, beta_posterior)
print(f"95% Credible Interval: [{p_L_informative:.4f}, {p_U_informative:.4f}]")

# Compute 99% credible interval
p_L_99_informative = beta_dist.ppf(0.005, alpha_posterior, beta_posterior)
p_U_99_informative = beta_dist.ppf(0.995, alpha_posterior, beta_posterior)
print(f"99% Credible Interval: [{p_L_99_informative:.4f}, {p_U_99_informative:.4f}]")

In [ ]:
# Visualize prior vs posterior
x_range = np.linspace(0, 0.5, 1000)
prior_pdf = beta_dist.pdf(x_range, alpha_prior, beta_prior)
posterior_pdf = beta_dist.pdf(x_range, alpha_posterior, beta_posterior)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_range, prior_pdf, 'b--', lw=2, label=f'Prior: Beta({alpha_prior:.1f}, {beta_prior:.1f})')
ax.plot(x_range, posterior_pdf, 'r-', lw=2, label=f'Posterior: Beta({alpha_posterior:.1f}, {beta_posterior:.1f})')

# Mark the non-inferiority boundary
ax.axvline(control_group_conversion_rate - epsilon, color='k', ls=':', lw=1.5, 
           label=f'Non-inferiority boundary = {control_group_conversion_rate - epsilon:.2f}')

# Mark the control conversion rate
ax.axvline(control_group_conversion_rate, color='g', ls=':', lw=1.5, 
           label=f'Control conversion rate = {control_group_conversion_rate:.2f}')

# Shade the 95% credible interval
mask = (x_range >= p_L_informative) & (x_range <= p_U_informative)
ax.fill_between(x_range[mask], posterior_pdf[mask], alpha=0.3, color='red', 
                label='95% Credible Interval')

ax.set_xlabel('Conversion rate p_A', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Informative Prior vs Posterior Distribution', fontsize=14)
ax.legend(loc='best')
ax.grid(True, ls=':', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Compute probabilityy that variant is non-inferior (p_A > p_C - epsilon)
# This is P(p_A > 0.17) under the posterior
prob_non_inferior = 1 - beta_dist.cdf(control_group_conversion_rate - epsilon, 
                                       alpha_posterior, beta_posterior)
print(f"Probability that variant is non-inferior: {prob_non_inferior:.4f}")
print(f"This means there's a {prob_non_inferior*100:.2f}% probabilityy that the variant conversion rate is above {control_group_conversion_rate - epsilon:.2f}")